In [5]:
import pandas as pd

In [6]:
df = pd.read_excel("../../data/raw/pr_cre_pr.xlsx",sheet_name="Tabla_2")
df_pib_pc = df.melt(id_vars="Comunidad Autónoma",var_name="año",value_name="pib_pc")
df_pib_pc["Comunidad Autónoma"] = df_pib_pc["Comunidad Autónoma"].str.title().str.replace(r",.*$", "", regex=True).str.strip()

In [7]:
df = pd.read_excel("../../data/raw/rentahogd24.xlsx",sheet_name="Tabla_2")
df_renta_pc = df.melt(id_vars="Comunidad Autónoma",var_name="año",value_name="renta_pc")
df_renta_pc["Comunidad Autónoma"] = df_renta_pc["Comunidad Autónoma"].str.title().str.replace(r",.*$", "", regex=True).str.strip()

In [8]:
merge = df_pib_pc.merge(df_renta_pc,on=["Comunidad Autónoma","año"],how="left")

In [9]:
df = pd.read_csv("../../data/raw/poblacion.csv", sep= ";", dtype=str)
df = df.drop(columns= "Total Nacional")
df = df[~df["Comunidades y Ciudades Autónomas"].isna()]
df = df[df["Edad (año a año)"] == "TOTAL EDADES"]
df = df[df["Españoles/Extranjeros"] == "Españoles"]
df = df[df["Sexo"] == "Ambos sexos"]
df = df.drop(columns=["Edad (año a año)","Españoles/Extranjeros","Sexo"])
df = df.rename(columns={"Comunidades y Ciudades Autónomas":"Comunidad Autónoma","Periodo":"año","Total":"poblacion"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df = df[df["año"]>"1999"]
df["año"] = df["año"].astype(int) 
df_poblacion = df.copy()
df_poblacion.año.dtypes
df_poblacion.poblacion = df_poblacion.poblacion.str.replace(".","")

In [10]:
merge1 = merge.merge(df_poblacion,on=["Comunidad Autónoma","año"],how="left")

In [11]:
df = pd.read_csv("../../data/raw/gasto_elevado_vivienda.csv", sep=";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año" , "Total":"gasto_elevado_vivienda(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_elevado_vivienda = df.copy()

In [12]:
merge2 = merge1.merge(df_elevado_vivienda,on=["Comunidad Autónoma","año"],how="left")

In [13]:
df = pd.read_csv("../../data/raw/falta_espacio_vivienda.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año" , "Total":"falta_espacio_vivienda(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_falta_espacio_vivienda = df.copy()

In [14]:
merge3 = merge2.merge(df_falta_espacio_vivienda,on=["Comunidad Autónoma","año"],how="left")

In [15]:
df = pd.read_csv("../../data/raw/Retrasos_pagos.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año" , "Total":"retrasos_pagos(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_retraso_pago = df.copy()

In [16]:
merge4 = merge3.merge(df_retraso_pago,on=["Comunidad Autónoma","año"],how="left")

In [17]:
df = pd.read_csv("../../data/raw/Renta_media_mediana.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()

In [18]:
df_renta_wide = (df
    .pivot_table(index=["Comunidad Autónoma", "año"],
                 columns="Renta media y mediana",
                 values="Total",
                 aggfunc="first")
    .reset_index()
)

In [19]:
df_renta_wide = df_renta_wide.rename(columns={"Renta media por unidad de consumo":"renta_media","Renta mediana por unidad de consumo":"renta_mediana"})

In [20]:
merge5 = merge4.merge(df_renta_wide,on=["Comunidad Autónoma","año"],how="left")

In [21]:
df = pd.read_csv("../../data/raw/Tasa_riesgo_pobreza.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"riesgo_pobreza(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_riesgo_pobreza = df.copy()

In [22]:
merge6 = merge5.merge(df_riesgo_pobreza,on=["Comunidad Autónoma","año"],how="left")
merge6["año"] = merge6["año"].astype(int)

In [23]:
df = pd.read_csv("../../data/raw/Dificultades_llegar_fin_mes.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"dificultad_fin_mes(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df["año"] = df["año"].astype(int)
df["dificultad_fin_mes(%)"] = df["dificultad_fin_mes(%)"].str.replace(",",".").astype(float)

In [24]:
df_dificultad = (df.groupby(["año","Comunidad Autónoma"],as_index=False)["dificultad_fin_mes(%)"].sum())

In [25]:
merge7 = merge6.merge(df_dificultad,on=["Comunidad Autónoma","año"],how="left")

In [26]:
df = pd.read_csv("../../data/raw/Desigualdad_S80S20.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"desigualdad_ing(S80/S20)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_desig_ingresos = df.copy()

In [27]:
merge8 = merge7.merge(df_desig_ingresos,on=["Comunidad Autónoma","año"],how="left")

In [28]:
df = pd.read_csv("../../data/raw/gastos_imprevistos.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"inc_gastos_imprevistos(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_imprevistos = df.copy()

In [29]:
merge9 = merge8.merge(df_imprevistos,on=["Comunidad Autónoma","año"],how="left")

In [30]:
df = pd.read_csv("../../data/raw/tasa_homicidio_criminalidad.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"tasa_criminalidad"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df = df[df["Indicador de calidad de vida"] == "Tasa de criminalidad"]
df = df.drop(columns= "Indicador de calidad de vida")
df_criminalidad = df.copy()

In [31]:
df= merge9.merge(df_criminalidad,on=["Comunidad Autónoma","año"],how="left")
df_periodo = df[(df["año"] > 2007) & (df["año"] < 2023)].copy()

In [32]:
df_periodo.to_csv("../../data/processed/pib_vs_calidad_vida.csv", index=False)